In [1]:
import subprocess
import pandas as pd
import re
import json
import seaborn as sns 
print(sns.__version__) # should be 0.9

import matplotlib.pyplot as plt
sns.set(style="whitegrid")


0.9.0


In [6]:
# Compile files
subprocess.run(["chmod", "+x" "compile.sh"])
subprocess.run(["./compile.sh"])


CompletedProcess(args=['./compile.sh'], returncode=0)

In [56]:
### For some reason the below only works for me with SphereEvaluation, not BentCigarFunction... Very annoying
### So just run the command and pipe it to a text file and read that out...

# result = subprocess.run(['java', '-jar', 'testrun.jar', '-evaluation=BentCigarFunction', '-seed=1', '-submission=player50'], stdout=subprocess.PIPE)
# output = result.stdout.decode('utf-8')
# print(result, result.stderr, result.stdout)
# print(output)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [10]:
import os


def create_run_df(run_output, run_config):
    df = pd.DataFrame()
    generation_nr = 0
    matches = re.finditer("Statistic({.+})", run_output)
    for m in matches:
        json_str = m[1]
        obj = pd.read_json(json_str, orient='column', typ='series')
        obj.at['run_config'] = run_config
        obj.at['generation_nr'] = generation_nr

        df = df.append(obj, ignore_index=True)
        generation_nr += 1
        
    return df

def create_all_runs_dfs(file_path):
    f = open(file_path, "r")
    output = f.read()
    run = 0
    for m in re.finditer("---Run ({.+})", output):
        print("Processing run {}".format(run))
#         print(m.start(), m.end())

        configuration_json = m.groups(1)[0]
        end_m = re.search("---Finished run---", output[m.start():])

#         print(end_m.start())

        run_output = output[m.end():end_m.end() + m.end()]

        df = create_run_df(run_output, configuration_json)

        run += 1
        yield df

# dfs = create_all_runs_dfs("bent_cigar.txt")
dfs = create_all_runs_dfs("./log/bent_cigar.txt")
df = next(dfs)


for next_df in dfs:
    total_df = df.append(next_df)
    


Processing run 0
Processing run 1


In [11]:
total_df.groupby(['run_config', 'generation_nr']).mean()

averageAge  \
run_config                                         generation_nr               
{"parentScheme": uniform, "recombinationScheme"... 0.0                   1.0   
                                                   1.0                   2.0   
                                                   2.0                   3.0   
                                                   3.0                   4.0   
                                                   4.0                   5.0   
                                                   5.0                   6.0   
                                                   6.0                   7.0   
                                                   7.0                   8.0   
                                                   8.0                   9.0   
                                                   9.0                  10.0   
                                                   10.0                 11.0   
                                                   11.0                 12.0   
                                                   12.0                 13.0   
                                                   13.0                 14.0   
                                                   14.0                 15.0   
                                                   15.0                 16.0   
                                                   16.0                 17.0   
                                                   17.0                 18.0   
                                                   18.0                 19.0   
                                                   19.0                 20.0   
                                                   20.0                 21.0   
                                                   21.0                 22.0   
                                                   22.0                 23.0   
                                                   23.0                 24.0   
                                                   24.0                 25.0   
                                                   25.0                 26.0   
                                                   26.0                 27.0   
                                                   27.0                 28.0   
                                                   28.0                 29.0   
                                                   29.0                 30.0   
...                                                                      ...   
                                                   71.0                 72.0   
                                                   72.0                 73.0   
                                                   73.0                 74.0   
                                                   74.0                 75.0   
                                                   75.0                 76.0   
                                                   76.0                 77.0   
                                                   77.0                 78.0   
                                                   78.0                 79.0   
                                                   79.0                 80.0   
                                                   80.0                 81.0   
                                                   81.0                 82.0   
                                                   82.0                 83.0   
                                                   83.0                 84.0   
                                                   84.0                 85.0   
                                                   85.0                 86.0   
                                                   86.0                 87.0   
                                                   87.0                 88.0   
                                                   88.0           

In [ ]:
df

In [ ]:
for col in df:
    print(col)
    if col != 'stdevGenomes':
        plt.figure(figsize=(10, 5))
        sns.scatterplot(y=df[col], x=range(len(df[col])))

    plt.show()



In [ ]:
# sns.pairplot(df)

# plt.show()